In [1]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
import math
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
config = {
    "mathtext.fontset":'stix',
    "font.family":'serif',
    "font.serif": ['Arial'],
    "font.size": 24,
    'axes.unicode_minus': False 
}
plt.rcParams.update(config)
plt.rcParams['axes.unicode_minus'] = False  
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (8, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = False

In [ ]:
small_data = pd.read_csv('/data/single_perovskite_oxide.csv')
small_data = small_data.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_data = small_data.drop(columns = 'composition').reset_index(drop=True)
small_data['composition'] = small_data['formula'].map(Composition)
y_eg_abo3 = small_data['target']

In [3]:
small_data

,formula,target,A,B,composition
0,SmCrO3,1.736200,Sm,Cr,"(Sm, Cr, O)"
1,ScCoO3,0.786000,Sc,Co,"(Sc, Co, O)"
2,GdCrO3,0.825000,Gd,Cr,"(Gd, Cr, O)"
3,YCrO3,1.767900,Y,Cr,"(Y, Cr, O)"
4,LaFeO3,0.848400,La,Fe,"(La, Fe, O)"
5,SmCoO3,0.803900,Sm,Co,"(Sm, Co, O)"
6,YFeO3,0.637300,Y,Fe,"(Y, Fe, O)"
7,LaCrO3,1.903900,La,Cr,"(La, Cr, O)"
8,PrCrO3,1.847000,Pr,Cr,"(Pr, Cr, O)"
9,NdCrO3,1.904000,Nd,Cr,"(Nd, Cr, O)"


In [ ]:
re_data = pd.read_csv('/data/all_features_df.csv')
ABO3_atomic_features = pd.read_csv('/data/small_atomic_features_new.csv').drop(columns = 'Unnamed: 0')
atomic_features = pd.read_csv('/data/ao_features.csv')

In [5]:
ABO3_atomic_features

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_p_neg_1,...,X_atomic_r_val_1,X_X,X_atomic_ie_1,X_atomic_ie_2,X_atomic_npunf,X_atomic_npval,X_atomic_nsval,X_atomic_nunf,X_atomic_nval,X_atomic_pol
0,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
1,-4.38669,-2.25963,-2.25963,-2.25963,1.578750,6.198956,1.68613,7.08536,1.84895,0.64805,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
2,-2.01997,-2.01997,-0.47011,-0.47011,0.846499,4.172046,2.68397,6.53600,1.94775,1.37595,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
3,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
4,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
5,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
6,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
7,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
8,-3.55283,-1.85729,-1.85729,-1.85729,-2.664785,2.129405,0.60005,6.41133,2.09365,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
9,-2.78148,-2.78148,-1.07232,-1.07232,2.903783,6.083156,1.87232,6.37620,2.11275,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802


In [6]:
atomic_labels = atomic_features.columns
select_atomic_labels = atomic_labels
atomic_features = atomic_features[select_atomic_labels]
atomic_features

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.766510,16.485700,5.0,5.0,0.0,0.0,6.0,6.0,4.650,11.600
1,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.881010,17.084000,3.0,7.0,0.0,0.0,3.0,9.0,5.100,7.500
2,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.902400,16.187700,4.0,6.0,0.0,0.0,4.0,8.0,4.930,8.400
3,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.758850,14.000000,6.0,4.0,0.0,0.0,7.0,5.0,4.050,15.700
4,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.343920,14.632200,0.0,10.0,4.0,2.0,4.0,14.0,4.150,7.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72843,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.925526,11.667650,8.7,1.3,0.0,0.0,17.8,4.0,3.366,25.100
72844,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.676880,13.749545,6.3,3.7,1.5,0.3,16.9,6.7,3.456,23.156
72845,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.322594,12.498752,6.3,0.7,1.2,0.6,16.6,4.0,3.636,22.379
72846,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.141961,0.000000,8.4,1.6,0.0,0.0,17.5,8.5,3.441,24.650


# Mgapie

In [7]:
synthetic_data = re_data[['formula','Eg']]
synthetic_data

,formula,Eg
0,Sm0.2Sc0.8Cr1O3,2.146025
1,Sm0.2Sc0.8Fe1O3,1.374682
2,Sm0.2Zr1Sc0.8O3,4.177025
3,Sm0.2Sc0.8Ge1O3,3.941249
4,Sm0.2Hf1Sc0.8O3,4.529786
...,...,...
35320,Sr1Ce0.7Zr0.3O3,3.843865
35321,Sr1Ce0.7Be0.3O3,3.431016
35322,Sr1Ce0.7Ge0.3O3,2.085930
35323,Sr1Ce0.7Hf0.3O3,3.646532


In [12]:
synthetic_data['composition'] = synthetic_data['formula'].map(Composition)
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen.core import Composition
feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
feature_labels = feature_calculators.feature_labels()
magpie_features = feature_calculators.featurize_dataframe(synthetic_data,col_id='composition')
magpie_features

MultipleFeaturizer:   0%|          | 0/35325 [00:00<?, ?it/s]

,formula,Eg,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,Sm0.2Sc0.8Cr1O3,2.146025,"(Sm, Sc, Cr, O)",8.0,62.0,54.0,15.44,8.9280,8.0,11.0,...,90.68,94.416,12.0,1.8,2.40,1.16,0.24,True,0.724240,0.147721
1,Sm0.2Sc0.8Fe1O3,1.374682,"(Sm, Sc, Fe, O)",8.0,62.0,54.0,15.84,9.4080,8.0,11.0,...,90.68,94.416,12.0,2.0,2.40,1.36,0.24,True,0.724240,0.140666
2,Sm0.2Zr1Sc0.8O3,4.177025,"(Sm, Zr, Sc, O)",8.0,62.0,54.0,18.64,12.7680,8.0,11.0,...,83.68,86.016,12.0,2.0,2.40,0.56,0.24,False,0.724240,0.161521
3,Sm0.2Sc0.8Ge1O3,3.941249,"(Sm, Sc, Ge, O)",8.0,62.0,54.0,17.04,10.8480,8.0,11.0,...,89.88,93.456,12.0,2.0,2.80,2.16,0.24,False,0.724240,0.133420
4,Sm0.2Hf1Sc0.8O3,4.529786,"(Sm, Hf, Sc, O)",8.0,72.0,64.0,25.04,21.7408,8.0,11.0,...,83.68,86.016,12.0,2.0,2.40,0.56,3.04,False,0.724240,0.162762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,Sr1Ce0.7Zr0.3O3,3.843865,"(Sr, Ce, Zr, O)",8.0,58.0,50.0,22.92,17.9040,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.26,0.14,True,0.787757,0.181550
35321,Sr1Ce0.7Be0.3O3,3.431016,"(Sr, Ce, Be, O)",4.0,58.0,54.0,20.76,17.3232,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.14,0.14,False,0.787757,0.179356
35322,Sr1Ce0.7Ge0.3O3,2.085930,"(Sr, Ce, Ge, O)",8.0,58.0,50.0,22.44,17.3280,8.0,8.0,...,92.86,97.032,12.0,2.0,2.52,0.74,0.14,True,0.787757,0.175717
35323,Sr1Ce0.7Hf0.3O3,3.646532,"(Sr, Ce, Hf, O)",8.0,72.0,64.0,24.84,20.2080,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.26,0.98,True,0.787757,0.181833


In [13]:
# magpie_features.to_csv('magpie_synthetic_data.csv')

In [14]:
small_magpie_features = feature_calculators.featurize_dataframe(small_data,col_id='composition')
small_magpie_features

MultipleFeaturizer:   0%|          | 0/52 [00:00<?, ?it/s]

,formula,target,A,B,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,SmCrO3,1.736200,Sm,Cr,"(Sm, Cr, O)",8.0,62.0,54.0,22.0,16.80,...,86.2,89.04,12.0,1.8,2.4,1.0,1.2,True,0.724240,0.154892
1,ScCoO3,0.786000,Sc,Co,"(Sc, Co, O)",8.0,27.0,19.0,14.4,7.68,...,84.8,87.36,12.0,2.0,2.4,1.6,0.0,True,0.660947,0.136622
2,GdCrO3,0.825000,Gd,Cr,"(Gd, Cr, O)",8.0,64.0,56.0,22.4,17.28,...,91.8,95.76,12.0,1.8,2.4,1.2,1.4,True,0.714753,0.153484
3,YCrO3,1.767900,Y,Cr,"(Y, Cr, O)",8.0,39.0,31.0,17.4,11.28,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.708321,0.152541
4,LaFeO3,0.848400,La,Fe,"(La, Fe, O)",8.0,57.0,49.0,21.4,16.08,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.745613,0.151694
5,SmCoO3,0.803900,Sm,Co,"(Sm, Co, O)",8.0,62.0,54.0,22.6,17.52,...,79.2,80.64,12.0,2.0,2.4,1.4,1.2,True,0.724240,0.146339
6,YFeO3,0.637300,Y,Fe,"(Y, Fe, O)",8.0,39.0,31.0,17.8,11.76,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.708321,0.145782
7,LaCrO3,1.903900,La,Cr,"(La, Cr, O)",8.0,57.0,49.0,21.0,15.60,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.745613,0.158143
8,PrCrO3,1.847000,Pr,Cr,"(Pr, Cr, O)",8.0,59.0,51.0,21.4,16.08,...,91.8,95.76,12.0,1.8,2.4,1.0,0.6,True,0.736585,0.156756
9,NdCrO3,1.904000,Nd,Cr,"(Nd, Cr, O)",8.0,60.0,52.0,21.6,16.32,...,91.8,95.76,12.0,1.8,2.4,1.0,0.8,True,0.733532,0.156291


# EC  + MGAPIE

In [20]:
data = re_data[select_atomic_labels].copy(deep = True)
data[feature_labels] = magpie_features[feature_labels]
data

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,90.68,94.416,12.0,1.8,2.40,1.16,0.24,True,0.724240,0.147721
1,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,90.68,94.416,12.0,2.0,2.40,1.36,0.24,True,0.724240,0.140666
2,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,83.68,86.016,12.0,2.0,2.40,0.56,0.24,False,0.724240,0.161521
3,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,89.88,93.456,12.0,2.0,2.80,2.16,0.24,False,0.724240,0.133420
4,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,83.68,86.016,12.0,2.0,2.40,0.56,3.04,False,0.724240,0.162762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,91.00,94.800,12.0,2.0,2.40,0.26,0.14,True,0.787757,0.181550
35321,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,91.00,94.800,12.0,2.0,2.40,0.14,0.14,False,0.787757,0.179356
35322,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,92.86,97.032,12.0,2.0,2.52,0.74,0.14,True,0.787757,0.175717
35323,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,91.00,94.800,12.0,2.0,2.40,0.26,0.98,True,0.787757,0.181833


In [35]:
# data.to_csv('ec_magpie_data.csv')

# Model

In [23]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

In [24]:
X = data.copy(deep = True)
y = synthetic_data['Eg']
X = remove_collinear_features(X,y,0.8)
fl = X.columns
X = data[fl]
X

,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_r_val,A_atomic_r_s_1,A_atomic_r_d_1,A_atomic_r_val_1,A_atomic_ie_2,...,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg d valence electrons,avg f valence electrons,compound possible
0,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.0,12.0,229.0,217.0,90.68,94.416,12.0,1.16,0.24,True
1,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.0,12.0,229.0,217.0,90.68,94.416,12.0,1.36,0.24,True
2,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.0,12.0,194.0,182.0,83.68,86.016,12.0,0.56,0.24,False
3,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.0,12.0,225.0,213.0,89.88,93.456,12.0,2.16,0.24,False
4,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.0,12.0,194.0,182.0,83.68,86.016,12.0,0.56,3.04,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.26,0.14,True
35321,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.14,0.14,False
35322,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,0.0,12.0,225.0,213.0,92.86,97.032,12.0,0.74,0.14,True
35323,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.26,0.98,True


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [26]:
# dop_magpie_etr = ExtraTreesRegressor(n_jobs = -1 ,random_state=42)
# dop_magpie_etr.fit(X_train, y_train)
# y_pred = dop_magpie_etr.predict(X_test)
# print('Performance on the test set: MAE = %0.4f' % mean_absolute_error(y_test,y_pred))
# print('Performance on the test set: RMSE = %0.4f' % np.sqrt(mean_squared_error(y_test, y_pred)))
# print('Performance on the test set: R2 SCORE = %0.4f' % r2_score(y_test,y_pred))

Performance on the test set: MAE = 0.3119
Performance on the test set: RMSE = 0.4283
Performance on the test set: R2 SCORE = 0.8804


In [27]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor(n_jobs=-1, random_state=42)
etr.fit(X_train, y_train)
feature_importance=pd.DataFrame()
feature_importance['features'] = X.columns
feature_importance['feature_importances'] = etr.feature_importances_
threshold = np.sort(feature_importance['feature_importances'].values)
score = []
n_features = []
cols = []
for i in tqdm(threshold):
    selector = SelectFromModel(etr,threshold=i)
    X_embedded = selector.fit_transform(X_train, y_train)
    once = cross_val_score(etr,X_embedded,y_train,cv=5,scoring="r2",n_jobs=-1).mean()#交叉验证
    score.append(once)
    n_features.append(X_embedded.shape[1])
    cols.append(X.columns[selector.get_support()])
result = pd.DataFrame({"col":cols,"score":score,"n_features":n_features})
result

100%|██████████| 108/108 [07:13<00:00,  4.02s/it]


,col,score,n_features
0,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.867235,108
1,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.867235,108
2,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.867235,108
3,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.867235,108
4,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.867235,108
...,...,...,...
103,"Index(['A_atomic_ndval', 'B_atomic_r_val', 'B_...",0.653128,5
104,"Index(['A_atomic_ndval', 'B_atomic_r_val', 'B_...",0.640478,4
105,"Index(['A_atomic_ndval', 'B_atomic_r_val', 'Ma...",0.573984,3
106,"Index(['B_atomic_r_val', 'MagpieData maximum N...",0.518415,2


In [28]:
features_select = result.sort_values(by='score',ascending=True)
features_select = features_select[features_select['score']>=0.86]

In [42]:
optimize_feature_labels = features_select.iloc[0]['col']
X_train = X.loc[X_train.index][optimize_feature_labels]
X_test = X.loc[X_test.index][optimize_feature_labels]
if 'O' in X_train.columns:
    X_train = X_train.drop(columns='O')
    X_test = X_test.drop(columns='O')
X_train.shape, X_test.shape

((28260, 38), (7065, 38))

In [30]:
X_train.columns

Index(['A_atomic_ea', 'A_atomic_r_val_1', 'A_atomic_ie_2', 'A_atomic_ndval',
       'A_atomic_nval', 'B_atomic_hfomo', 'B_atomic_lfumo', 'B_atomic_lpumo',
       'B_atomic_ip_by_half_charged_homo', 'B_atomic_r_val_neg_1',
       'B_atomic_r_val', 'B_atomic_r_val_05', 'B_atomic_r_d_1',
       'B_atomic_ie_2', 'B_atomic_ndval', 'B_atomic_npval', 'B_atomic_nunf',
       'B_atomic_phi', 'MagpieData mean MeltingT',
       'MagpieData avg_dev MeltingT', 'MagpieData range Column',
       'MagpieData maximum NdValence', 'MagpieData range NdValence',
       'MagpieData minimum NValence', 'MagpieData avg_dev NsUnfilled',
       'MagpieData avg_dev NpUnfilled', 'MagpieData maximum NUnfilled',
       'MagpieData mean NUnfilled', 'MagpieData mean GSbandgap',
       'MagpieData avg_dev GSbandgap', 'MagpieData mean GSmagmom',
       'MagpieData avg_dev GSmagmom', 'MagpieData maximum SpaceGroupNumber',
       'MagpieData range SpaceGroupNumber', 'MagpieData mean SpaceGroupNumber',
       'MagpieData a

In [33]:
ABO3_data = ABO3_atomic_features[select_atomic_labels]
ABO3_data[feature_labels] = small_magpie_features[feature_labels]
ABO3_data

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,86.2,89.04,12.0,1.8,2.4,1.0,1.2,True,0.724240,0.154892
1,-4.38669,-2.25963,-2.25963,-2.25963,1.578750,6.198956,1.68613,7.08536,1.84895,0.64805,...,84.8,87.36,12.0,2.0,2.4,1.6,0.0,True,0.660947,0.136622
2,-2.01997,-2.01997,-0.47011,-0.47011,0.846499,4.172046,2.68397,6.53600,1.94775,1.37595,...,91.8,95.76,12.0,1.8,2.4,1.2,1.4,True,0.714753,0.153484
3,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.708321,0.152541
4,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.745613,0.151694
5,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,79.2,80.64,12.0,2.0,2.4,1.4,1.2,True,0.724240,0.146339
6,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.708321,0.145782
7,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.745613,0.158143
8,-3.55283,-1.85729,-1.85729,-1.85729,-2.664785,2.129405,0.60005,6.41133,2.09365,0.34455,...,91.8,95.76,12.0,1.8,2.4,1.0,0.6,True,0.736585,0.156756
9,-2.78148,-2.78148,-1.07232,-1.07232,2.903783,6.083156,1.87232,6.37620,2.11275,1.31275,...,91.8,95.76,12.0,1.8,2.4,1.0,0.8,True,0.733532,0.156291


In [36]:
ABO3_data.to_csv('ABO3_ec_magpie_data.csv')

In [43]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm import tqdm
train_model = ExtraTreesRegressor(n_jobs = -1, random_state = 42)
y_eg_abo3 = small_data['target']
error1 = []
error2 = []
r2_prm = []
error1_base=[]
error2_base=[]
r2_prl=[]
train_set = []
test_set = []
tmp_df = pd.DataFrame()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_indices = kf.split(X,y)
base_features = ABO3_data[optimize_feature_labels]
for i,(train_index, test_index) in enumerate(cv_indices):
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    X_train = X_train[optimize_feature_labels]
    X_test = X_test[optimize_feature_labels]
    name = f"fold_{i}"
    train_model.fit(X_train, y_train)
    y_pred =  train_model.predict(X_test)
    mae_model = mean_absolute_error(y_test, y_pred)
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_model = r2_score(y_test,y_pred)
    error1.append(mae_model)
    error2.append(rmse_model)
    r2_prm.append(r2_model)
    
    y_base = train_model.predict(base_features)
    tmp_df[f'column_{i}'] = y_base
    MAE_BASE = mean_absolute_error(y_base,y_eg_abo3)
    RMSE_BASE = np.sqrt(mean_squared_error(y_base,y_eg_abo3))
    R2_BASE = r2_score(y_base,y_eg_abo3)
    error1_base.append(MAE_BASE)
    error2_base.append(RMSE_BASE)
    r2_prl.append(R2_BASE)
    
    print(name)
    print(f"Performance on the test set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (mae_model, rmse_model, r2_model))
    print(f"Performance on the small set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (MAE_BASE, RMSE_BASE, R2_BASE))
    
mean_error1 = sum(error1) / len(error1)
mean_error2 = sum(error2) / len(error2)
mean_r2 = sum(r2_prm) / len(r2_prm)
print('Performance on the test set using 5-fold: MAE = %0.4f' % mean_error1)
print('Performance on the test set using 5-fold: RMSE = %0.4f' % mean_error2)
print('Performance on the test set using 5-fold: R2 = %0.4f' % mean_r2)

mean_error1_base = sum(error1_base) / len(error1_base)
mean_error2_base = sum(error2_base) / len(error2_base)
mean_r2_base = sum(r2_prl) / len(r2_prl)
print('Performance on the valid set using 5-fold: MAE = %0.4f' % mean_error1_base)
print('Performance on the valid set using 5-fold: RMSE = %0.4f' % mean_error2_base)
print('Performance on the valid set using 5-fold: R2 = %0.4f' % mean_r2_base)

fold_0
Performance on the test set: MAE = 0.3173,RMSE = 0.4384, R2 = 0.8747
Performance on the small set: MAE = 0.4826,RMSE = 0.6114, R2 = 0.5697
fold_1
Performance on the test set: MAE = 0.3264,RMSE = 0.4501, R2 = 0.8669
Performance on the small set: MAE = 0.4708,RMSE = 0.5959, R2 = 0.5949
fold_2
Performance on the test set: MAE = 0.3252,RMSE = 0.4490, R2 = 0.8720
Performance on the small set: MAE = 0.4592,RMSE = 0.5974, R2 = 0.5929
fold_3
Performance on the test set: MAE = 0.3192,RMSE = 0.4411, R2 = 0.8715
Performance on the small set: MAE = 0.4701,RMSE = 0.6109, R2 = 0.5844
fold_4
Performance on the test set: MAE = 0.3153,RMSE = 0.4346, R2 = 0.8758
Performance on the small set: MAE = 0.4714,RMSE = 0.5999, R2 = 0.5980
Performance on the test set using 5-fold: MAE = 0.3207
Performance on the test set using 5-fold: RMSE = 0.4426
Performance on the test set using 5-fold: R2 = 0.8722
Performance on the valid set using 5-fold: MAE = 0.4708
Performance on the valid set using 5-fold: RMSE =

In [44]:
best_train_index = train_set[4]
best_test_index = test_set[4]
best_X_train = X.loc[best_train_index].reset_index(drop=True)
best_X_test = X.loc[best_test_index].reset_index(drop=True)
best_y_train = y.loc[best_train_index].reset_index(drop=True)
best_y_test = y.loc[best_test_index].reset_index(drop=True)

In [45]:
best_X_train = best_X_train[optimize_feature_labels]
best_X_test = best_X_test[optimize_feature_labels]

In [52]:
model_best = ExtraTreesRegressor(n_jobs=-1, random_state=42)
model_best.fit(best_X_train, best_y_train)
y_pred = model_best.predict(best_X_test)
MAE_4 = mean_absolute_error(best_y_test,y_pred)
RMSE_4 = np.sqrt(mean_squared_error(best_y_test, y_pred))
R2_4 = r2_score(best_y_test,y_pred)
print('Performance on the test set: MAE = %0.4f' % MAE_4)
print('Performance on the test set: RMSE = %0.4f' % RMSE_4)
print('Performance on the test set: R2 SCORE = %0.4f' % R2_4)

Performance on the test set: MAE = 0.3153
Performance on the test set: RMSE = 0.4346
Performance on the test set: R2 SCORE = 0.8758


In [47]:
best_f = optimize_feature_labels.tolist()

In [48]:
# import json
# with open("best_feature_m_ec.json", "w") as f:
#     json.dump(best_f, f)

In [53]:
import pickle
with open('model_m_ec.pkl', 'wb') as f:
    pickle.dump(model_best, f)